In [7]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from functions import *

# 1. Importar data sets

In [8]:
# Cargar los datasets
df_final_demo = pd.read_csv("df_final_demo.txt")
df_final_experiment_clients = pd.read_csv("df_final_experiment_clients.txt")
df_final_web_data_pt_1 = pd.read_csv("df_final_web_data_pt_1.txt")
df_final_web_data_pt_2 = pd.read_csv("df_final_web_data_pt_2.txt")


# 2. Limpieza y union de data sets 

In [9]:
# Limpiar los datos
df_demo_clean = clean_demo_data(df_final_demo)
df_experiment_clean = clean_experiment_clients_data(df_final_experiment_clients)
df_web_clean = clean_web_data(df_final_web_data_pt_1, df_final_web_data_pt_2)

# Unir los datasets limpios
df_final = merge_datasets(df_web_clean, df_experiment_clean, df_demo_clean)

In [10]:
df_final.head(3)

,client_id,visitor_id,visit_id,process_step,date_time,is_complete,time_before_step,time_diff_step,prev_process_step,is_error,Variation,clnt_tenure_yr,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,bal_category,account_since_child
0,555,402506806_56087378777,637149525_38041617439_716659,start,2017-04-15 12:57:56,1,NaT,NaN,NaN,False,Test,3,29,U,2,25454.66,2,6,bajo,no
1,555,402506806_56087378777,637149525_38041617439_716659,step_1,2017-04-15 12:58:03,1,2017-04-15 12:57:56,7.0,start,False,Test,3,29,U,2,25454.66,2,6,bajo,no
2,555,402506806_56087378777,637149525_38041617439_716659,step_2,2017-04-15 12:58:35,1,2017-04-15 12:58:03,32.0,step_1,False,Test,3,29,U,2,25454.66,2,6,bajo,no


In [11]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 318614 entries, 0 to 318613
Data columns (total 20 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   client_id            318614 non-null  int64         
 1   visitor_id           318614 non-null  object        
 2   visit_id             318614 non-null  object        
 3   process_step         318614 non-null  category      
 4   date_time            318614 non-null  datetime64[ns]
 5   is_complete          318614 non-null  int64         
 6   time_before_step     249751 non-null  datetime64[ns]
 7   time_diff_step       249751 non-null  float64       
 8   prev_process_step    249751 non-null  category      
 9   is_error             318614 non-null  bool          
 10  Variation            318614 non-null  object        
 11  clnt_tenure_yr       318614 non-null  int32         
 12  clnt_age             318614 non-null  int32         
 13  gendr         

# 3. KPIs

## 3.1 Tasa de finalización

In [12]:
completion_stats = calculate_completion_stats(df_final)
completion_stats

,Variation,Completed Clients,Incomplete Clients,Completion Rate (%),Incompletion Rate (%)
0,general,33807,23762,58.724313,41.275687
1,Control,15303,11952,56.147496,43.852504
2,Test,18504,11810,61.041103,38.958897


## 3.2 Tiempo en cada paso

In [13]:
time_stats = calculate_time_stats(df_final)
time_stats

,Variation,process_step,mean_time,min_time,max_time,num_entries
0,Control,start,151.886076,0.0,40235.0,16590
1,Control,step_1,43.037307,0.0,4150.0,29137
2,Control,step_2,38.770338,0.0,1724.0,25555
3,Control,step_3,92.975700,0.0,3635.0,22428
4,Control,confirm,129.046178,0.0,9396.0,16588
5,Test,start,147.305936,0.0,24819.0,23263
6,Test,step_1,37.667716,0.0,3636.0,38115
7,Test,step_2,48.276693,0.0,1858.0,30561
8,Test,step_3,96.897236,0.0,1923.0,25398
9,Test,confirm,129.194520,0.0,14581.0,22116


## 3.3 Tasa de errores

In [14]:
error_rate_stats = calculate_error_rate(df_final)
error_rate_stats

,Group,Total Records,Total Errors,Error Rate (%)
0,Control,142286,9545,6.71
1,Test,176328,16138,9.15


# 4. Prueba de hipotesis

**1.4 Hipótesis de tasa de finalización**
Dado que el nuevo diseño (grupo de prueba) tuvo una tasa de finalización más alta en comparación con el diseño anterior (grupo de control), debe confirmar si esta diferencia es estadísticamente significativa.

**H0: La media de tasa de finalización entre grupos es igual**

**H1: La media de tasa de finalización entre grupos es diferente**

In [15]:
hypothesis_testing_completion(df_final, alpha=0.05)

{'z_statistic': 8.754041342564873,
 'p_value': 1.0292281822361728e-18,
 'decision': 'Rechaza la hipótesis nula: hay una diferencia significativa en las tasas de finalización.'}

**2.4 Hipótesis umbral costo - efectividad**

Umbral : Vanguard ha establecido este aumento mínimo en la tasa de finalización en un 5 % . Se estima que esta es la tasa a la que los beneficios proyectados, en términos de mayor participación de los usuarios y posibles ingresos, superarán los costos del nuevo diseño.

Se le solicita que realice otro análisis para asegurarse de que el aumento observado en la tasa de finalización de la prueba A/B cumpla o supere este umbral del 5 % . Si el nuevo diseño no genera al menos este nivel de mejora, es posible que no se justifique desde una perspectiva de costos, independientemente de su importancia estadística.

**H0: El aumento de la tasa de finalización no alcanza el umbral del 5%**

**H1: El aumento en la tasa de finalización es significativo y cumple el umbral del 5%**

In [16]:
hypothesis_testing_with_threshold(df_final, cost_effectiveness_threshold=0.05, alpha=0.05)

{'z_statistic': -159.46225476798637,
 'p_value': 1.0,
 'decision': 'No se rechaza la hipótesis nula: el aumento en la tasa de completación no cumple con el umbral del 5%.'}

**3.4 Hipotesis diferencias de tiempos por pasos**

Se realiza una prueba de hipótesis para comparar las medias de duración por paso entre los grupos Control y Test

**H0: La media de duuracion por pasos entre los grupos son iguales**

**H1: La media de duracion por pasos entre los grupos son diferentes**

In [17]:
hypothesis_test_time_stats(df_final, alpha=0.05)

,process_step,Control Mean Time,Test Mean Time,T-Statistic,P-Value,Alpha,Decision
0,start,151.89,147.31,1.0066,0.3142,0.05,No se rechaza H₀
1,step_1,43.04,37.67,6.9414,0.0000,0.05,Se rechaza H₀
2,step_2,38.77,48.28,-13.4010,0.0000,0.05,Se rechaza H₀
3,step_3,92.98,96.90,-3.5658,0.0004,0.05,Se rechaza H₀
4,confirm,129.05,129.19,-0.0668,0.9467,0.05,No se rechaza H₀


**3.5 Hipotesis diferencias en tasas de error**

Se realiza una prueba de hipótesis para comparar las tasas de error entre los grupos Control y Test

**H0: La tasa de error entre los grupos son iguales**

**H1: La tasa entre los grupos son diferentes**

In [18]:
hypothesis_test_error_rate(df_final, alpha=0.05)

{'Control Error Rate (%)': 6.71,
 'Test Error Rate (%)': 9.15,
 'Z-Statistic': -25.1918,
 'P-Value': 0.0,
 'Alpha': 0.05,
 'Decision': 'Se rechaza H₀',
 'Explanation': 'Las diferencias en las tasas de error son estadísticamente significativas'}

**3.6 Hipotesis Edad por grupos**

Evaluar si ha existido diferencia entre las edades por grupo 

**H0 : No hubo diferencia entre las edades de los grupos**

**H1 : Si hubo diferencia entre las edades de los grupos**

In [19]:
df_test = df_final[df_final['Variation'] == 'Test']
df_control = df_final[df_final['Variation'] == 'Control']
hypothesis_testing(df_test, df_control, "clnt_age", alpha=0.05)

{'t_statistic': -0.3368883107328185,
 'p_value': 0.7362364795738359,
 'decision': 'No se rechaza la hipótesis nula: no hay una diferencia significativa entre los grupos.'}

**3.7 Hipotesis Tiempo de permanencia por grupos**

Evaluar si ha existido diferencia entre las edades por grupo 

**H0 : No hubo diferencia entre el tiempo de permanencia de los grupos**

**H1 : Si hubo diferencia entre el tiempo de permanencia de los grupos**

In [20]:
hypothesis_testing(df_test, df_control, "clnt_tenure_yr", alpha=0.05)

{'t_statistic': -0.3754411672192049,
 'p_value': 0.7073722458536124,
 'decision': 'No se rechaza la hipótesis nula: no hay una diferencia significativa entre los grupos.'}